# Initial Experimentation

## Web References

### Bitcoin Address

- [Bitcoin Address](https://learnmeabitcoin.com/technical/address)
- [Bitcoin Checksum](https://learnmeabitcoin.com/technical/checksum)
- [How do I check the checksum of a Bitcoin address?](https://bitcoin.stackexchange.com/questions/32353/how-do-i-check-the-checksum-of-a-bitcoin-address)
- [Base58Check encoding](https://en.bitcoin.it/wiki/Base58Check_encoding#Creating_a_Base58Check_string)

In [144]:
import unittest
from cryptography.hazmat.primitives import hashes

## Transaction Class

In [145]:
class TestTransaction(unittest.TestCase):
    def setUp(self):
        self.tx = Transaction(
            sender_hash=bytearray(20),
            recipient_hash=bytearray(20),
            sender_public_key=hashes.Hash(hashes.SHA256()),
            amount=1,
            fee=1,
            nonce=1,
            signature=None,
            txid=None)

    def test_verify_sender_hash(self):
        """
        Test that the sender_hash is verified correctly.
        """
        # test a valid hash
        self.tx.sender_hash = bytearray(20)
        self.tx.verify(
            sender_balance=1,
            sender_previous_nonce=1)            

        # test an invalid hash
        self.tx.sender_hash = bytearray(b'thequickbrownfox')
        with self.assertRaises(ValueError):
            self.tx.verify(
                sender_balance=1,
                sender_previous_nonce=1)

    def test_verify_recipient_hash(self):
        """
        Test that the recipient_hash is verified correctly.
        """
        # test a valid hash
        self.tx.recipient_hash = bytearray(20)
        self.tx.verify(
            sender_balance=1,
            sender_previous_nonce=1)            

        # test an invalid hash
        self.tx.recipient_hash = bytearray(b'thequickbrownfox')
        with self.assertRaises(ValueError):
            self.tx.verify(
                sender_balance=1,
                sender_previous_nonce=1)

    def test_verify_amount(self):
        """
        Test that the amount is verified correctly.
        """
        # test a valid hash
        self.tx.amount = 50
        self.tx.verify(
            sender_balance=100,
            sender_previous_nonce=1)     

        # the amount should be a whole number
        self.tx.amount = 50.1
        with self.assertRaises(ValueError):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=1)

        # the amount should be between 1 and sender_balance
        self.tx.amount = 0
        with self.assertRaises(ValueError):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=1)

        self.tx.amount = -10
        with self.assertRaises(ValueError):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=1)   

        self.tx.amount = 101
        with self.assertRaises(ValueError):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=1)                               

In [146]:
class Transaction:
    def __init__(self, sender_hash:bytes, recipient_hash:bytes, sender_public_key:bytes, amount:int, fee:int, nonce:int, signature, txid):
        self.sender_hash = sender_hash
        self.recipient_hash = recipient_hash
        self.sender_public_key = sender_public_key
        self.amount = amount
        self.fee = fee
        self.nonce = nonce
        self.signature = signature
        self.txid = txid
        #self.hash = self.calculate_hash()

    def verify(self, sender_balance:int, sender_previous_nonce:int):
        if len(self.sender_hash) != 20:
            raise ValueError('Invalid sender hash')
        if len(self.recipient_hash) != 20:
            raise ValueError('Invalid recipient hash')
        if self.amount < 1:
            raise ValueError('Invalid amount')
        if self.amount > sender_balance:
            raise ValueError('Insufficient funds')
        if float(self.amount).is_integer() == False:
            raise ValueError('The amount should be a whole number')

unittest.main(argv=['ignored', '-v'], exit=False)

test_verify_amount (__main__.TestTransaction)
Test that the amount is verified correctly. ... ERROR
test_verify_recipient_hash (__main__.TestTransaction)
Test that the recipient_hash is verified correctly. ... ERROR
test_verify_sender_hash (__main__.TestTransaction)
Test that the sender_hash is verified correctly. ... ERROR

ERROR: test_verify_amount (__main__.TestTransaction)
Test that the amount is verified correctly.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_19813/804464372.py", line 53, in test_verify_amount
    self.tx.verify(
  File "/tmp/ipykernel_19813/1072120782.py", line 22, in verify
    if self.amount.is_integer() == False:
AttributeError: 'int' object has no attribute 'is_integer'

ERROR: test_verify_recipient_hash (__main__.TestTransaction)
Test that the recipient_hash is verified correctly.
----------------------------------------------------------------------
Traceback (most recent ca

In [147]:
a_bytes = bytes('a', 'utf-8')
type(a_bytes)

bytes